In [1]:
import numpy as np

# Define the grid world as a 2D numpy array
grid_world = np.array([[0, 1, -1],
                       [0, 0, 0],
                       [0, float('inf'), 0],
                       [float('inf'), 0, 0]])

# Define the actions (up, down, left, right)
actions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

# Define the transition probabilities
prob_action = 0.7
prob_perpendicular = 0.15

# Define the rewards and penalties
reward_goal = 1
penalty_red = -1
step_cost = -0.04

# Define the discount factor
discount_factor = 0.95

# Initialize the utility values
num_rows, num_cols = grid_world.shape
utilities = np.zeros((num_rows, num_cols))

# Helper function to check if a cell is valid (i.e., not a wall or out of bounds)
def is_valid_cell(row, col):
    return row >= 0 and row < num_rows and col >= 0 and col < num_cols and grid_world[row][col] != float('inf')

# Value Iteration algorithm
while True:
    # Initialize the delta for convergence check
    delta = 0
    
    # Update the utility values for each cell
    for row in range(num_rows):
        for col in range(num_cols):
            # Skip walls and goal state
            if grid_world[row][col] == float('inf') or grid_world[row][col] == reward_goal:
                continue
            
            # Get the current utility value
            current_utility = utilities[row][col]
            
            # Initialize the maximum utility value for the current cell
            max_utility = float('-inf')
            
            # Try each action and compute the expected utility value
            for action in actions:
                # Compute the next state after taking the action
                next_row = row + action[0]
                next_col = col + action[1]
                
                # Check if the next state is valid
                if is_valid_cell(next_row, next_col):
                    # Compute the expected utility value for the current action
                    expected_utility = prob_action * utilities[next_row][next_col]
                    
                    # Compute the expected utility values for the two perpendicular actions
                    for perpendicular_action in actions:
                        if perpendicular_action != action:
                            perpendicular_row = row + perpendicular_action[0]
                            perpendicular_col = col + perpendicular_action[1]
                            if is_valid_cell(perpendicular_row, perpendicular_col):
                                expected_utility += prob_perpendicular * utilities[perpendicular_row][perpendicular_col]
                    
                    # Update the maximum utility value
                    max_utility = max(max_utility, expected_utility)
            
            # Update the utility value for the current cell
            utilities[row][col] = grid_world[row][col] + discount_factor * max_utility
            
            # Update the delta for convergence check
            delta = max(delta, abs(current_utility - utilities[row][col]))
    
    # Print the current utility values
    print("Utility Values after Iteration:")
    print(utilities)
    print()
    
    # Check for convergence
    if delta <= 0.0001:
        break

# Compute the best policy for each cell
# Compute the best policy for each cell
best_policy = np.zeros((num_rows, num_cols), dtype=str)
for row in range(num_rows):
    for col in range(num_cols):
        # Skip walls
        if grid_world[row][col] == float('inf'):
            best_policy[row][col] = 'WALL'
            continue

        # Skip goal state
        if grid_world[row][col] == reward_goal:
            best_policy[row][col] = 'GOAL'
            continue

        # Initialize the maximum expected utility and best action
        max_expected_utility = float('-inf')
        best_action = None

        # Try each action and compute the expected utility value
        for action_idx, action in enumerate(actions):
            # Compute the next state after taking the action
            next_row = row + action[0]
            next_col = col + action[1]

            # Check if the next state is valid
            if is_valid_cell(next_row, next_col):
                # Compute the expected utility value for the current action
                expected_utility = prob_action * utilities[next_row][next_col]

                # Compute the expected utility values for the two perpendicular actions
                for perpendicular_action in actions:
                    if perpendicular_action != action:
                        perpendicular_row = row + perpendicular_action[0]
                        perpendicular_col = col + perpendicular_action[1]
                        if is_valid_cell(perpendicular_row, perpendicular_col):
                            expected_utility += prob_perpendicular * utilities[perpendicular_row][perpendicular_col]

                # Update the maximum expected utility and best action
                if expected_utility > max_expected_utility:
                    max_expected_utility = expected_utility
                    best_action = action_idx

        # Map the best action to a direction for the current cell
        if best_action is not None:
            if best_action == 0:
                best_policy[row][col] = 'UP'
            elif best_action == 1:
                best_policy[row][col] = 'DOWN'
            elif best_action == 2:
                best_policy[row][col] = 'LEFT'
            elif best_action == 3:
                best_policy[row][col] = 'RIGHT'

# Print the best policy
print("Best Policy after Convergence:")
for row in range(num_rows):
    for col in range(num_cols):
        print(best_policy[row][col], end='\t')
    print()



Utility Values after Iteration:
[[ 0.          0.         -1.        ]
 [ 0.          0.         -0.1425    ]
 [ 0.          0.         -0.02030625]
 [ 0.          0.         -0.00289364]]

Utility Values after Iteration:
[[ 0.          0.         -1.02030625]
 [ 0.         -0.02030625 -0.16179094]
 [ 0.          0.         -0.02497948]
 [ 0.         -0.00192427 -0.00483922]]

Utility Values after Iteration:
[[ 0.          0.         -1.02305521]
 [-0.00289364 -0.02346755 -0.16495087]
 [-0.00192427  0.         -0.02672358]
 [ 0.         -0.00321808 -0.00594813]]

Utility Values after Iteration:
[[-4.12343789e-04  0.00000000e+00 -1.02350550e+00]
 [-3.89254345e-03 -2.40601858e-02 -1.65657667e-01]
 [-2.58854140e-03  0.00000000e+00 -2.75617253e-02]
 [ 0.00000000e+00 -3.95550781e-03 -6.55795855e-03]]

Utility Values after Iteration:
[[-5.54687442e-04  0.00000000e+00 -1.02360622e+00]
 [-4.16631077e-03 -2.41999168e-02 -1.65884377e-01]
 [-2.77059666e-03  0.00000000e+00 -2.79995661e-02]
 [ 0.00